# Nesse notebook tem códigos para aprender a rodar um modelo varias vezes por grupos.

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


In [5]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [6]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 71.92%


In [7]:
# usando um outro seed e deu um resultado maior que outro seed
# então o fator de decisão está na aleatoriedade e isso não é bom
# precisamos amenizar o efeito disso
# e se rodarmos o treino uma vez, não é recomendado, temos que rodar variáveis 
# vezes para tirar a media, desvio padrão ou um intervalo para ter um valor consideravel
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)
# podemos separar os nossos dados em "n" pedaços e pegar umas partes para treino e teste
# depois é só inverter ou trocar as partes, fazendo treino e teste com todas as partes
# cruzando essa validação com as partes.
# isso chama cross validation

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


In [8]:
from sklearn.model_selection import cross_validate

# modelo de arvore
modelo = DecisionTreeClassifier(max_depth=2)
# cv = 3, quantos pedaços quero que quebre os dados
results = cross_validate(modelo, x, y, cv = 3)
results

{'fit_time': array([0.00789165, 0.00757003, 0.00722861]),
 'score_time': array([0.00213075, 0.00253916, 0.00195432]),
 'test_score': array([0.75704859, 0.7629763 , 0.75337534])}

In [9]:
# tira o retorno do treino
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)

In [10]:
# o resultado do treino, como temos 3 notas, então a nossa base foi separada e rodou 3 vezes
results['test_score']

array([0.75704859, 0.7629763 , 0.75337534])

In [11]:
media = results ['test_score'].mean()

In [12]:
# criando o intervalo de desvio padrão
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy %.2f %.2f" % (media - 2 * desvio_padrao, media + 2 * desvio_padrao))

Accuracy 0.75 0.77


In [13]:
from sklearn.model_selection import cross_validate

# mudamos o seed e o intervalo continua o mesmo, isso é bom, tiramos o fator
# aleatoriedade, mas ainda estamos presos ao cv = 3, pois quem mandou escolher 3?
# quem pediu para repartir a base em 3 partes? 
SEED = 153
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [74.99, 76.57]


In [14]:
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99, 76.57]


In [15]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
# na literatura indica um cv de 5 ou 10, então por padrão vamos usar o 5
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# Aleatoriedade no cross validate

In [16]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [17]:
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

# passando o parametro shuffle, eu embaralho as partes, tipo, a gente pede para
# separar a base de dados em 10 partes e pegar os dados dessas 10 partes e embaralhar
# entre si, isso serve para testar novamente os resultados dos nossos dados sobre
# o efeito da aleatoriedade
cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]


In [18]:
# Simular situação horrível de azar
# Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.
# pois o kfold não tem o parâmetro de colocar os dados proporcional para repartição dos dados

dados.sort_values("vendido", ascending=True)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [19]:
# separando os atributos 
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [20]:
# rodadno sem o shuffle, que embaralha as partes
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [21]:
# olha como o shuffle da uma boa diferença
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.30, 79.26]


In [22]:
# o stratifiedkfod ele tem por padrão a função de fazer a proporção de dados
# na separação por partes da base e no embaralhamento
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.55, 78.01]


In [23]:
# aqui é só uma diferença do stratifiedkfolg com e sem o shuffle, mostrando que
# a proporção que o stratifiedkfold faz, tem valor aproximado com o shuffle
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

# é muito importante separar bem os dados de treino e teste, não é bom o dado A
# ir pro treino e tambem ir pro teste, a boa prática é que os dados de teste
#  algortimo nunca tenha visto, pois se os dados de teste são os mesmo que o de treino
# o algortimo vai saber resolver de letra, mas o proposito é ele testar com dados
# que ele nunca viu e com base nos dados de teste, resolver o problema dos dados
# novos.

Accuracy médio: 75.78
Accuracy intervalo: [73.83, 77.73]


In [24]:
np.random.seed(SEED)
np.random.randint(-2, 3)

-2

In [25]:
np.random.seed(SEED)
np.random.randint(-2, 3, size=10000)

array([-2,  2,  0, ...,  0,  2,  2])

In [26]:
# criando uma coluna para modelo de carros, usando numeros aleatórios com base
# na idade do carro, pois um cara que tem 18 anos de idade, o modelo dele vai ser 
# um intervalo por exemplo de 16 a 20 anos de idade.
# então criamos um random com 10000 mil numeros entre -2 a 2 e somamos com a idade do modelo
# e adicionamos uma coluna 
np.random.seed(SEED)
dados["modelo_aleatorio"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [27]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

In [28]:
dados.modelo_aleatorio.min()

-1

In [29]:
abs(dados.modelo_aleatorio.min())

1

In [30]:
# como temos um modelo -1 que parece estranho, deixamos ele de forma absoluta 
# tirando o negativo dele
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min())
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,23
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,5
4,117384.68,1,4,12770.11290,4


In [31]:
dados.modelo_aleatorio.min()

0

In [42]:
# o modelo de carro inicial é 1
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,24,16
1,40557.96,1,20,12622.05362,30,22
2,89627.50,0,12,11440.79806,20,12
3,95276.14,0,3,43167.32682,12,4
4,117384.68,1,4,12770.11290,11,3


In [43]:
dados.modelo_aleatorio.unique()

array([24, 30, 20, 12, 11, 19, 26, 25, 21,  8, 23, 18, 17, 22,  9, 13, 27,
       29, 16, 15, 28, 14, 10,  7])

In [59]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,48,18
1,40557.96,1,20,12622.05362,54,24
2,89627.50,0,12,11440.79806,44,14
3,95276.14,0,3,43167.32682,36,6
4,117384.68,1,4,12770.11290,35,5


In [50]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [51]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

In [52]:
# testando o estimador e é muito importante escolhe o treino e teste ideal
# o stratified ele ignora o agrupamento, então os agrupamentos de modelo que aparecer
# no treino, vai aparecer no teste, por isso que o resultado é bom
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.55, 78.01]


In [54]:
# o kfold é a mesma situação, ele usa os dados de treino para o teste
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [61]:
# Quando utilizamos o GroupKFold, tem que deixar definidos os grupos que gostaria de agrupar e a coluna do grupo.
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)
# a acurraci ta bem próximo do stratified, isso acontece pq os carros em si
# são similares entre si, mesmo separando os dados de treino e os dados para teste
# os dado ainda tem uma certa similaridade, agora, se aparecer um carro totalmente
# diferente, um carro voador, creio que o resultado seria diferente

Accuracy médio: 75.78
Accuracy intervalo: [73.67, 77.90]


In [62]:
# O algoritmo de arvore de decisão, não precisamos padronizar os atributos, mas
# para outros algoritmos temos que padronizar, pois são muito sensiveis a escala
# dos dados
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# o scaler n usa seed, mas o svc sim
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
# padronizando as variaveis
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
# treinando o modelo com os dados padronizados
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi de %.2f %%" % acuracia)

A acurácia foi de 77.48 %


In [68]:
# cross validation
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
# o svc esta usando dados não padronizados
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 77.27
Accuracy intervalo: [74.35, 80.20]


In [64]:
# padronizando os dados
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [65]:
x_azar_escalado

array([[ 0.3636103 , -0.39591706,  0.24651152],
       [ 0.79214444, -0.18257669,  0.03346607],
       [ 0.72310419,  1.09746554,  1.66435195],
       ...,
       [ 0.17822399, -0.60925743, -0.73405662],
       [ 0.2248779 ,  0.45744443, -0.39741686],
       [-1.34266351,  0.88412517,  1.52099053]])

In [69]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.70
Accuracy intervalo: [74.30, 79.10]


In [73]:
# quando padronizamos as variaveis, a gente padronizou pra todos os dados
# e depois rodamos o groupkfold 10x, mas para cada rodada do groupkfold
# temos que refazer a padronização e rodar o modelo, então fica assim, 
# padronizamos e rodamos o modelo, voltamos e padronizamos dnv, rodamos o modelo
# e vai pra segunda rodada do group e assim por diante, isso é um processo 
# e o pipeline nos ajuda com isso
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

# agora para cada um dos processos, ele vai rodar a padronização e o modelo, até dar 10 rodadas
# veja que os valores são similares aos antigos modelos, mas os antigos estavam errados
# pois rodamos uma vez o scaler para 10 treinos e isso não pode, temos que rodar
# n scaler para n treinos

Accuracy médio: 76.70
Accuracy intervalo: [74.30, 79.10]
